# Getting Started with TwelveLabs

This notebook provides a complete workflow example for using the TwelveLabs API, from creating an index to uploading videos and performing searches.

## Overview
This notebook demonstrates the end-to-end process:
1. **Create an Index** - Set up a container for your video content
2. **Upload Videos** - Add videos to your index
3. **Monitor Indexing** - Track the indexing process
4. **Search Videos** - Query your indexed videos using natural language
5. **Process Results** - Extract and use search results

## Prerequisites
- TwelveLabs API key (stored in environment variable `TL_API_KEY`)
- Required Python packages: `twelvelabs`, `python-dotenv`

In [ ]:
# Import required libraries for TwelveLabs API interaction
import uuid  # For generating unique identifiers if needed
from twelvelabs import TwelveLabs  # Main TwelveLabs client class
from twelvelabs.indexes import IndexesCreateRequestModelsItem  # Type hint for index model configuration
from twelvelabs.tasks import TasksRetrieveResponse  # Type hint for task response objects

d:\projects\12labs\new1\.venv\Lib\site-packages\twelvelabs\core\pydantic_utilities.py:13: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.datetime_parse import parse_date as parse_date


## Configure Your API Key

Load your TwelveLabs API key from environment variables. Make sure you have a `.env` file with `TL_API_KEY` set.

In [ ]:
# Load environment variables from .env file
import os
from dotenv import load_dotenv  

# Load environment variables (including API key)
load_dotenv()

# Retrieve the TwelveLabs API key from environment variables
TL_API_KEY = os.getenv("TL_API_KEY")

## Step 1: Create an Index

An index is a container that organizes your video content. It defines which AI models will be used to process and understand your videos.

In [ ]:
# Initialize the TwelveLabs client with your API key
# This client will be used for all API interactions
client = TwelveLabs(api_key=TL_API_KEY)

# Create an index - a container for organizing your video content
# The index defines which AI models will process your videos
index = client.indexes.create(
    index_name="test_index_1",  # Name for your index
    models=[
        IndexesCreateRequestModelsItem(
            model_name="marengo3.0",  # Advanced video understanding model
            model_options=["visual", "audio"]  # Process both visual and audio content
        )
    ]
)

# Verify that the index was created successfully
if index.id is None:
    raise RuntimeError("Failed to create an index.")

# Print the index ID - you'll need this for future operations
print(f"Created index: id={index.id}")

Created index: id=6952ac44e5676b2af30d5330


## Step 2: Upload Videos to Your Index

Upload a video file to your index. The video will be processed asynchronously. You'll receive a task ID to monitor the indexing progress.

In [ ]:
# Upload a video to your index
# This creates an indexing task that processes the video asynchronously
task = client.tasks.create(
    index_id=index.id,  # ID of the index to upload to
    # Option 1: Upload from a local file
    video_file=open("D:/projects/12labs/new1/videos/Red_Bull_Aizawl _2024.mp4", "rb")
    # 'rb' mode opens file in binary read mode, required for file uploads
    
    # Option 2: Upload from a URL (uncomment to use)
    # video_url="https://example.com/video.mp4"
)

# Print the task ID - use this to monitor the indexing progress
print(f"Created task: id={task.id}")

Created task: id=6952ad025dc27e4008d6d67a


In [ ]:
# Monitor the indexing process
# This function will be called each time the task status updates
def on_task_update(task: TasksRetrieveResponse):
    print(f"  Status={task.status}")  # Print current status (e.g., "validating", "indexing", "ready")

# Wait for the indexing task to complete
# This blocks until the task is done (ready or failed)
task = client.tasks.wait_for_done(task_id=task.id, callback=on_task_update)

# Verify that indexing completed successfully
if task.status != "ready":
    raise RuntimeError(f"Indexing failed with status {task.status}")

# Print the video ID - this is the unique identifier for your uploaded video
# You'll use this video_id for searching and analysis operations
print(f"Upload complete. The unique identifier of your video is {task.video_id}.")


Upload complete. The unique identifier of your video is 6952ad025dc27e4008d6d67a.


In [ ]:
# Perform a search request on your indexed videos
# Search using natural language queries - the AI understands what you're looking for
Querry = "introduction"  # Your search query (can be any natural language description)

# Execute the search query
# The search looks through all videos in the specified index
search_pager = client.search.query(
    index_id=index.id,  # ID of the index to search in
    query_text=Querry,  # Natural language search query
    search_options=["visual", "audio"],  # Search in both visual and audio content
)


In [ ]:
# Process and display the search results
# The search returns video clips (segments) that match your query
print("Search results:")
for clip in search_pager:
    # Each clip represents a segment of a video that matches your query
    print(
        f" video_id {clip.video_id} rank={clip.rank} start={clip.start} end={clip.end}"
        # - video_id: ID of the video containing this clip
        # - rank: Relevance ranking (lower = more relevant)
        # - start: Start time of the clip in seconds
        # - end: End time of the clip in seconds
    )


Search results:
 video_id 6952ad025dc27e4008d6d67a rank=1 start=132.5 end=139.25
 video_id 6952ad025dc27e4008d6d67a rank=2 start=0.0 end=7.0
 video_id 6952ad025dc27e4008d6d67a rank=3 start=7.0 end=14.0
 video_id 6952ad025dc27e4008d6d67a rank=4 start=14.0 end=21.0
 video_id 6952ad025dc27e4008d6d67a rank=5 start=21.0 end=28.0
 video_id 6952ad025dc27e4008d6d67a rank=6 start=125.75 end=132.5
 video_id 6952ad025dc27e4008d6d67a rank=7 start=42.0 end=49.0
 video_id 6952ad025dc27e4008d6d67a rank=8 start=77.0 end=84.0
 video_id 6952ad025dc27e4008d6d67a rank=9 start=35.0 end=42.0
 video_id 6952ad025dc27e4008d6d67a rank=10 start=56.0 end=63.0
 video_id 6952ad025dc27e4008d6d67a rank=11 start=84.0 end=98.0
